In [1]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import wavfile

Using TensorFlow backend.


In [2]:
from keras.layers import Conv1D, MaxPool1D, Activation, Dense, Input, Flatten
import tensorflow as tf

# Prepare Training set

In [16]:
annotations_path = '../data/MagnaTagATune/annotation_reduced.csv'
annotations = pd.read_csv(annotations_path, sep='\t')
train_set, test_set = train_test_split(annotations['mp3_path'], train_size=0.005, test_size=0.001) 

In [19]:
x_train = []
x_test = []
for value in tqdm(train_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_train.append(data)
x_train = np.array(x_train)[:,:,np.newaxis]

for value in tqdm(test_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_test.append(data)
x_test = np.array(x_test)[:,:,np.newaxis]

print("X train shape:",x_train.shape)
print("X test shape:",x_test.shape)

100%|██████████| 26/26 [00:00<00:00, 2146.65it/s]

X train shape: (129, 465984, 1)
X test shape: (26, 465984, 1)


In [21]:
y_train = annotations.loc[annotations['mp3_path'].isin(train_set)].drop(columns=['mp3_path','Unnamed: 0']).values
y_test = annotations.loc[annotations['mp3_path'].isin(test_set)].drop(columns=['mp3_path','Unnamed: 0']).values
print("Y train shape:",y_train.shape)
print("Y test shape:",y_test.shape)

Y train shape: (129, 40)
Y test shape: (26, 40)


In [22]:
config = tf.ConfigProto(device_count={'GPU':0})
#config.gpu_options.allow_growth = True
session = tf.Session(config=config)
keras.backend.set_session(session)

model = keras.Sequential()
model.add(Conv1D(filters=10, kernel_size=13, strides=6, input_shape=(x_train.shape[1],1)))
model.add(MaxPool1D(pool_size=3)) #
model.add(Conv1D(filters=4, kernel_size=3, strides=1))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=y_train.shape[1], activation='sigmoid'))

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 77662, 10)         140       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 25887, 10)         0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 25885, 4)          124       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 12942, 4)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 51768)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 40)                2070760   
Total params: 2,071,024
Trainable params: 2,071,024
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [25]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
129/129 [==============================] - 3s 27ms/step - loss: 5.7567
Epoch 2/10
129/129 [==============================] - 3s 25ms/step - loss: 3.5617
Epoch 3/10
129/129 [==============================] - 3s 25ms/step - loss: 2.9657
Epoch 4/10
129/129 [==============================] - 3s 25ms/step - loss: 2.5486
Epoch 5/10
129/129 [==============================] - 3s 25ms/step - loss: 2.2568
Epoch 6/10
129/129 [==============================] - 3s 25ms/step - loss: 2.0482
Epoch 7/10
129/129 [==============================] - 3s 25ms/step - loss: 2.0451
Epoch 8/10
129/129 [==============================] - 3s 26ms/step - loss: 2.0248
Epoch 9/10
129/129 [==============================] - 3s 27ms/step - loss: 2.0049
Epoch 10/10
129/129 [==============================] - 3s 25ms/step - loss: 1.9802


In [27]:
predictions = model.predict(x_train)